In [42]:
import math
import matplotlib as plt
import numpy as np
%matplotlib inline

SyntaxError: invalid syntax (3941536859.py, line 2)

In [26]:
def f(x): 
    return 6*x**2 + 3* + 12

In [27]:
f(3)

90

In [41]:
h = 0.00001
x = 230
dx= ((f(x+h) - f(x))/h)
print(dx)
plt.plot(20, 10)

2760.0000612437725


AttributeError: module 'matplotlib' has no attribute 'plot'